# Modélisation en flot

## Indices

$n,h,d,r$ : nombre d'articles d'une commande, heure de la commande, délai de livraison, numéro de l'itinéraire


In [1]:
from itertools import chain, combinations
import pulp
from pyexcel_ods import get_data
import numpy as np
import copy
from datetime import datetime

In [2]:
H = range(24)
D = range(1,4)
L = ["CAR", "PFS", "Mag"]
N = range(1,10) # Nombre d'articles dans une commande

# Déclarations des données fixées
prix_car_Mag = 0
car_cutoff = 0
delai_car_Mag = 0
d_volumineuse = 0
n_total_cmd = 0
stocks = {}
stocks_volu = {}
prix_preparation = {} # utilisation : prix_preparation["PFS"][True] -> prix de préparation d'un article volumineux 

In [3]:
class Route:
    def __init__(self):
        self.possible = {}
        self.delai = 0
        self.cutoff = 24
        self.prix = {}
        self.volu = False
        self.depart_volu = "Mag"

    def prix_prepa(self, i, n, lieu):
        p_prepa = 0
        if(n == 0):
            return 0
        if(lieu == "PFS"):
            p_prepa = i * prix_preparation["PFS"][0] + prix_preparation["PFS"][1] * (self.volu and self.depart_volu == "PFS")
        elif(lieu == "Mag"):
            p_prepa = (n-i) * prix_preparation["Mag"][0] + prix_preparation["Mag"][1] * (self.volu and self.depart_volu == "Mag")
        else:#lieu == "CAR"
            p_prepa = prix_preparation["CAR"][1] * (self.volu and self.depart_volu == "CAR")
        return p_prepa

    def prix_total(self, i=0, n=0):
        total = self.prix_prepa(i,n,"PFS") + self.prix_prepa(i,n,"Mag") + self.prix_prepa(i,n,"CAR")
        for key in self.prix:
            total += self.prix[key]
        return total

    def est_possible(self, i, n, l="Mag"):
        if self.volu and l != self.depart_volu:
            return False
        if i == 0:
            return self.possible[0]
        if i == n:
            return self.possible["n"]
        return self.possible["i"]

    def to_str(self):
        if not self.volu:
            if self.possible[0]:
                return "Mag--->Client"
            if self.possible["i"]:
                return "PFS--->Mag--->Client"
            if self.possible["n"]:
                return "PFS--->Client"
        else:
            if self.possible[0]:
                if(self.depart_volu == "Mag"):
                    return "Mag-V->Client"
                if(self.depart_volu == "PFS"):
                    return "PFS-V->Mag-V->Client"
                if(self.depart_volu == "CAR"):
                    return "CAR-V->Mag-V->Client"

            if self.possible["i"]:
                if(self.depart_volu == "Mag"):
                    return "PFS--->Mag-V->Client"
                if(self.depart_volu == "PFS"):
                    return "PFS-V->Mag-V->Client"
                if(self.depart_volu == "CAR"):
                    return "\nPFS---|\n      |->Mag-V->Client \nCar-V-|               "

            if self.possible["n"]:
                if(self.depart_volu == "Mag"):
                    return "Mag-V->Client + PFS--->Client"
                if(self.depart_volu == "PFS"):
                    return "PFS-V->Client"
                if(self.depart_volu == "CAR"):
                    return "CAR-V->Mag-V->Client + PFS--->Client"

class Livraison:
    def __init__(self):
        self.prix = 0
        self.cutoff = 24
        self.delai = 0

class DecathlonProbleme:
    def __init__(self):
        self.lp = None
        self.listeRoute = []
        self.var_flot_std = []
        self.var_flot_volu = []
        self.demande_volu = []
        self.demande_std = []
        self.nb_total_std = 0
        self.nb_total_volu = 0

In [4]:
def read_file():
    filepath = "./data.ods"
    tableur = get_data(filepath)
    return tableur["Sheet1"]


## Données

$Delai_{route}, Cutoff_{route}, Possible_{route}, Possible^{volu}_{route}$ : Délai de l'itinéraire, cutoff de l'itinéraire, ensemble de couples $(i,n)$ tels que la route permet $(MP)_{i,n}$, ensemble des triplets $(i,n,l)$ tels que la route permet $(l+MP)_{i,n}$ 

$N_{cmd}$ : nombre total de commande

$D^{volu}$ : Proportion de demande volumineuse

$D^{volu}_{nhd}$ : Proportion des commandes volumineuse de taille $n$ à l'heure $h$ et avec un délai $d$

$D_{nhd}$ : Proportion des commandes standard de taille $n$ à l'heure $h$ et avec un délai $d$

$P_{i,n,r}$ et  $P^{volu}_{i,n,r}$ : Prix de livraison de $(MP)_{i,n}$ ou $(l+MP)_{i,n}$ avec l'itinéraire $r$

In [5]:
def get_data_from_file(R):
    data = read_file()
    prixPFSCol = 0
    prixMagCol = 1
    prixCol = 2
    cutoffCol = 3
    delaiCol = 4
    possible_col = 5
    index_offset = 1
    index = 0
    # Lecture des routes sans volumineux écrites dans le fichier excel
    while index_offset + index < len(data) and len(data[index_offset + index]) >= possible_col + 3 and data[index_offset + index][prixCol] != "":
        row = data[index_offset + index]
        r = Route()
        r.volu = False
        r.delai = row[delaiCol]
        r.cutoff = row[cutoffCol]
        r.prix = {"Mag": row[prixMagCol], "PFS": row[prixPFSCol]}
        r.possible = {0: row[possible_col],
                    "i": row[possible_col+1], "n": row[possible_col+2]}
        R.append(r)
        index += 1

    # Lecture des delta entre livraison non volumineuse et volumineuse (on identifie une livraison par son prix)
    delta_prix_vol = {}
    index = 0
    while len(data[index_offset + index]) >= 12 and data[index + index_offset][9] != '':
        if data[index + index_offset][11] != "":
            delta_prix_vol[data[index + index_offset]
                        [9]] = data[index + index_offset][11]
        index += 1


    # Liste des livraisons PFS vers Magasin volumineuses
    livraison_volu_PFS = []
    index = 0
    while len(data[index_offset + index]) >= 17 and data[index + index_offset][14] != '':
        livr = Livraison()
        livr.prix = data[index + index_offset][14]
        livr.cutoff = data[index + index_offset][15]
        livr.delai = data[index + index_offset][16]
        livraison_volu_PFS.append(livr)
        index += 1

    # Liste des livraisons Magasin vers client volumineuses
    livraison_volu_Mag = []
    index = 0
    while len(data[index_offset + index]) >= 21 and data[index + index_offset][18] != '':
        livr = Livraison()
        livr.prix = data[index + index_offset][18]
        livr.cutoff = data[index + index_offset][19]
        livr.delai = data[index + index_offset][20]
        livraison_volu_Mag.append(livr)
        index += 1


    # Génération des livraisons volumineuses à partir des non volumineuses, on ajoute le point de départ du colis volumineux
    for r in list(R):
        if (r.possible[0]):
            # PFS
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "PFS"
            if delta_prix_vol.get(r.prix["Mag"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                for l in livraison_volu_PFS:
                    route_pfs = copy.deepcopy(new_r)
                    route_pfs.prix["PFS"] = l.prix
                    route_pfs.delai += l.delai
                    route_pfs.cutoff = l.cutoff
                    R.append(route_pfs)

            # Mag
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "Mag"
            if delta_prix_vol.get(r.prix["Mag"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                R.append(new_r)

            # CAR
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "CAR"
            if delta_prix_vol.get(r.prix["Mag"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                new_r.prix["CAR"] = prix_car_Mag
                new_r.delai += 1
                new_r.cutoff = car_cutoff
                R.append(new_r)

        if (r.possible['i']):
            # PFS
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "PFS"
            if delta_prix_vol.get(r.prix["Mag"]) != None and delta_prix_vol.get(r.prix["PFS"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                new_r.prix["PFS"] += delta_prix_vol.get(r.prix["PFS"])
                R.append(new_r)

            # Mag
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "Mag"
            if delta_prix_vol.get(r.prix["Mag"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                new_r.possible["n"] = True
                R.append(new_r)

            # CAR
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "CAR"
            if delta_prix_vol.get(r.prix["Mag"]) != None:
                new_r.prix["Mag"] += delta_prix_vol.get(r.prix["Mag"])
                new_r.prix["CAR"] = prix_car_Mag
                new_r.possible["n"] = True
                new_r.cutoff = min(new_r.cutoff, car_cutoff) if new_r.cutoff == 14 else new_r.cutoff # hard coded stuff cutoff == 14 => PFS -> Mag is J+1   
                R.append(new_r)

        if (r.possible['n']):
            # PFS
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "PFS"
            if delta_prix_vol.get(r.prix["PFS"]) != None:
                new_r.prix["PFS"] += delta_prix_vol.get(r.prix["PFS"])
                R.append(new_r)

            # Mag
            new_r = copy.deepcopy(r)
            new_r.volu = True
            new_r.depart_volu = "Mag"
            for l in livraison_volu_PFS:
                route_Mag_pfs = copy.deepcopy(new_r)
                route_Mag_pfs.prix["Mag"] = l.prix
                route_Mag_pfs.delai = max(route_Mag_pfs.delai, l.delai)
                route_Mag_pfs.cutoff = min(route_Mag_pfs.cutoff, l.cutoff)
                R.append(route_Mag_pfs)


In [6]:
# Récupération des données Décathlon
def get_demande():
    # Pour l'instant on génère une demande constante
    demande = [[]]
    demande_volu = [[]]
    for n in N:
        demande.append([])
        demande_volu.append([])
        for h in H:
            demande[n].append([0])
            demande_volu[n].append([0])
            for d in D:
                demande[n][h].append(1 / (len(N) * len(H) * len(D)))
                demande_volu[n][h].append(1 / (len(N) * len(H) * len(D)))

    n_total_article_std = sum([n_total_cmd * (1-d_volumineuse) * demande[n][h][d] * n for n in N for h in H for d in D])
    n_total_article_std += sum([n_total_cmd * d_volumineuse * demande_volu[n][h][d] * n for n in N for h in H for d in D])
    n_articles_volumineux =  d_volumineuse * n_total_cmd
    return demande, demande_volu, n_total_article_std, n_articles_volumineux

## Variables de décisions

$f^{volu}_{inhdrl}$ : Proportion partant du nœud "n articles volumineux"  qui passe par l'arc $ (l + MP)_{i,n-1}$ et qui prend l'itinéraire $r$

$f_{inhdr}$ : proportion partant du nœud "n articles"  qui passe par l'arc $ (MP)_{i,n}$ et qui prend l'itinéraire $r$

## Variables intermédiaires

$q_{inhdr} = N_{cmd}  (1 - D^{volu}) \times D_{nhd} f_{inhdr}$ : Quantité de commandes passant par l'arc $ (MP)_{i,j}$ et qui prend l'itinéraire $r$

$q^{volu}_{inhdrl} = N_{cmd}  D^{volu} \times D^{volu}_{nhd} f^{volu}_{inhdrl}$ : quantité de commandes qui passe par l'arc $ (l + MP)_{i,n-1}$ et qui prend l'itinéraire $r$

In [7]:
def generer_probleme_et_variables(R):
    probleme = pulp.LpProblem("probleme_de_livraison_flux", pulp.LpMinimize)
    var_flux = []
    var_flux_volu = []
    for i in range(N[-1] + 1):
        var_flux.append([])
        var_flux_volu.append([])
        for n in range(i):
            var_flux[i].append([])
            var_flux_volu[i].append([])
        for n in range(i,N[-1] + 1):
            var_flux[i].append([])
            var_flux_volu[i].append([])
            if(i == 0 and n == 0):
                continue
            for h in H:
                var_flux[i][n].append([{}])
                var_flux_volu[i][n].append([{}])
                for d in D:
                    var_flux[i][n][h].append({})
                    var_flux_volu[i][n][h].append({})
                    for r in R:
                        if r.volu:
                            var_flux_volu[i][n][h][d][r] = {}
                            for l in L:
                                if(h >= r.cutoff or not r.est_possible(i,n,l) or d != r.delai):
                                    var_flux_volu[i][n][h][d][r][l] = 0
                                else:
                                    var_flux_volu[i][n][h][d][r][l] = \
                                        pulp.LpVariable("X_" + str(i) + "_" + str(n)+ "_" + str(h)+ "_" + str(d)+ "_" + str(R.index(r)) + "_" + l,0,1)
                        else:
                            if(h >= r.cutoff or not r.est_possible(i,n) or d != r.delai):
                                var_flux[i][n][h][d][r] = 0
                            else:
                                var_flux[i][n][h][d][r] = pulp.LpVariable("X_" + str(i) + "_" + str(n)+ "_" + str(h)+ "_" + str(d)+ "_" + str(R.index(r)),0,1)
    return probleme, var_flux, var_flux_volu


In [8]:
# Variables intermédiaires
def quantite_volu(pbDecat:DecathlonProbleme,i,n,h,d,r,l):
    if isinstance(pbDecat.var_flot_volu[i][n][h][d].get(r), type(None)):
        return 0
    return pbDecat.var_flot_volu[i][n][h][d][r][l] * n_total_cmd * d_volumineuse * pbDecat.demande_volu[n][h][d]

def quantite(pbDecat:DecathlonProbleme,i,n,h,d,r):
    if isinstance(pbDecat.var_flot_std[i][n][h][d].get(r), type(None)):
        return 0
    return pbDecat.var_flot_std[i][n][h][d][r] * n_total_cmd * (1-d_volumineuse) * pbDecat.demande_std[n][h][d]

## Fonctions de coût

$
C_{std} = \sum_{nhdr} \bigg [ q_{0nhdr} P_{Mag,Client,r} + q_{nnhdr} P_{PFS,Client,r} + \sum_{i=1}^{n-1} q_{inhdr} \left ( P_{PFS, Mag,r} + P_{Mag,Client,r} \right ) \bigg ] \\
C_{volu} = \sum_{nhdr} \bigg [ q^{volu}_{0nhdr,CAR} (P^{volu}_{Car,Mag,r} + P^{volu}_{Mag,Client,r}) + \sum_{i=1}^{n} q^{volu}_{inhdr,CAR} \left ( P^{volu}_{CAR, Mag,r} + P_{PFS,Mag,r} + P^{volu}_{Mag,Client,r} \right ) \\
     + q^{volu}_{nnhdr,PFS} P^{volu}_{PFS,Client,r} + \sum_{i=0}^{n-1} q^{volu}_{inhdr,PFS} \left ( P^{volu}_{PFS, Mag,r} + P^{volu}_{Mag,Client,r} \right )\\
     + q^{volu}_{0nhdr,Mag} P^{volu}_{Mag,Client,r} + \sum_{i=1}^{n} q^{volu}_{inhdr,Mag} \left ( P_{PFS,Mag,r} + P^{volu}_{Mag,Client,r} \right )
    \bigg ]
$

Étant donné que les quantités $q$ seront contraintes en fonction des itinéraires possibles, on peut réécrire :
$$ C_{std} = \sum_{nhdr} \sum_{i=0}^n q_{inhdr} Prix_{i,n,r}
\newline
C_{volu} = \sum_{nhdrl} \sum_{i=0}^n q^{volu}_{inhdrl} Prix_{i,n,r} $$

In [9]:
def cost_function(pbDecat:DecathlonProbleme):
    R = pbDecat.listeRoute
    cout_std = pulp.lpSum([quantite(pbDecat,i,n,h,d,r) * r.prix_total(i,n) for r in R if not r.volu for n in N for i in range(n+1) for h in H for d in D])
    cout_volu = pulp.lpSum([quantite_volu(pbDecat,i,n,h,d,r,l) * r.prix_total(i,n) for r in R if r.volu for n in N for i in range(n+1) for h in H for d in D for l in L])
    pbDecat.lp += cout_std + cout_volu

## Contrainte de capacitées

$\displaystyle C^{volu}_l \geq \sum_{inhdr} q^{volu}_{inhdrl} \ l\in[CAR,PFS,Mag]$

$\displaystyle C_{PFS} \geq \sum_{inhdr} i \left ( q_{inhdr} + \sum_l q^{volu}_{inhdrl} \right )$

$\displaystyle C_{Mag} \geq \sum_{inhdr} (n-i) \left ( q_{inhdr} + \sum_l q^{volu}_{inhdrl} \right )$

In [10]:
def contrainte_capacite(pbDecat:DecathlonProbleme):
    R = pbDecat.listeRoute
    for l in L:
        pbDecat.lp += pulp.lpSum([quantite_volu(pbDecat,i,n,h,d,r,l) for n in N for i in range(n+1) for h in H for d in D for r in R]) <= stocks_volu[l] * pbDecat.nb_total_volu
    global PFS_stock
    global Mag_stock
    PFS_stock = 0
    Mag_stock = 0
    for r in R:
        for d in D:
            for h in H :
                for n in N:
                    for i in range(n+1):
                        PFS_stock += i*(quantite(pbDecat,i,n,h,d,r) + quantite_volu(pbDecat,i,n,h,d,r,"PFS") + quantite_volu(pbDecat,i,n,h,d,r,"Mag") + quantite_volu(pbDecat,i,n,h,d,r,"CAR"))
                        Mag_stock += (n-i)*(quantite(pbDecat,i,n,h,d,r) + quantite_volu(pbDecat,i,n,h,d,r,"PFS") + quantite_volu(pbDecat,i,n,h,d,r,"Mag") + quantite_volu(pbDecat,i,n,h,d,r,"CAR"))
    pbDecat.lp += PFS_stock<= stocks["PFS"] * pbDecat.nb_total_std
    pbDecat.lp += Mag_stock<= stocks["Mag"] * pbDecat.nb_total_std

## Contraintes de réponse à la demande

$\forall (n,h,d)\ \sum_{i=0}^n \sum_r f_{inhdr} = 1$

$\forall (n,h,d)\ \sum_{i=0}^n \sum_r \sum_l f^{volu}_{inhdrl} = 1$


In [11]:
def contrainte_satisfaire_demande(pbDecat:DecathlonProbleme):
    R = pbDecat.listeRoute
    for n in N:
        for h in H:
            for d in D:
                pbDecat.lp += pulp.lpSum([ pbDecat.var_flot_std[i][n][h][d][r] for r in R if not r.volu for i in range(n+1)]) == 1
                pbDecat.lp += pulp.lpSum([ pbDecat.var_flot_volu[i][n][h][d][r][l] for r in R if r.volu for i in range(n+1) for l in L]) == 1

## Condition de prise des itinéraires

Cette condition est implémentée dans la déclaration des variables, la contrainte conditionne la déclaration de la variable $f_{inhdr}$ sinon c'est un $0$ 

$\forall (i,n,h,d,r)\ f_{inhdr} = 0 $ si $h \geq Cutoff_r $ ou $(i,n) \notin Possible_r$ ou $d \ne Delai_r$

$\forall (i,n,h,d,r,l)\ f^{volu}_{inhdrl} = 0 $ si $h \geq Cutoff_r $ ou $(i,n,l) \notin Possible^{volu}_r$ ou $d \ne Delai_r$

In [12]:
def write_to_file(pbDecat:DecathlonProbleme):
    R = pbDecat.listeRoute
    r_count = {}
    cout_total_prep = {"PFS":0, "Mag":0, "CAR":0}
    for r in R:
        r_count[r] = 0
    for n in N:
        for i in range(n + 1):
            for h in H:
                for d in D:
                    for r in R:
                        if r.volu:
                            for l in L:
                                if (pbDecat.var_flot_volu[i][n][h][d][r][l] != 0 and
                                        pbDecat.var_flot_volu[i][n][h][d][r][l].varValue !=
                                        0):
                                    r_count[r] += pulp.value(quantite_volu(pbDecat,i,n,h,d,r,l))
                                    for lieu in cout_total_prep:
                                        cout_total_prep[lieu] += r.prix_prepa(i,n,lieu) *  pulp.value(quantite_volu(pbDecat,i,n,h,d,r,l))
                        if not r.volu:
                            if (pbDecat.var_flot_std[i][n][h][d][r] != 0
                                    and pbDecat.var_flot_std[i][n][h][d][r].varValue != 0):
                                r_count[r] += pulp.value(quantite(pbDecat,i,n,h,d,r))
                                for lieu in cout_total_prep:
                                    cout_total_prep[lieu] += r.prix_prepa(i,n,lieu) * pulp.value(quantite(pbDecat,i,n,h,d,r)) 
    R.sort(key=lambda r:(r.to_str(), r.delai, r.cutoff))

    trajet_dict = {}
    key = []
    last_r = None
    for r in R:
        if r_count[r] != 0:
            for k in key:
                if trajet_dict.get(k) == None:
                    trajet_dict[k] = 0
                trajet_dict[k] += r_count[last_r]
            key = []
            last_r = r
            if not r.volu:
                if r.possible[0]:
                    key.append(("Mag", "Client", r.delai, False, r.prix["Mag"]))
                    continue
                if r.possible["i"]:
                    d_pfs = 1 if r.cutoff == 14 else 2
                    key.append(("Mag", "Client", r.delai - d_pfs, False, r.prix["Mag"]))
                    key.append(("PFS","Mag", d_pfs, False, r.cutoff, r.prix["PFS"]))
                    continue
                if r.possible["n"]:
                    key.append(("PFS","Client", r.delai, False, r.cutoff, r.prix["PFS"]))
                    continue
            else:
                if r.possible[0]:
                    if(r.depart_volu == "Mag"):
                        key.append(("Mag", "Client", r.delai, True, r.prix["Mag"]))
                        continue
                    if(r.depart_volu == "PFS"):
                        d_pfs = 1 if r.cutoff == 14 else 2
                        key.append(("Mag", "Client", r.delai - d_pfs, True, r.prix["Mag"]))
                        key.append(("PFS","Mag", d_pfs, True, r.cutoff, r.prix["PFS"]))
                        continue
                    if(r.depart_volu == "CAR"):
                        key.append(("Mag", "Client", r.delai - 1, True, r.prix["Mag"]))
                        key.append(("CAR","Mag", r.prix["CAR"]))
                        continue

                if r.possible["i"]:
                    if(r.depart_volu == "Mag"):
                        d_pfs = 1 if r.cutoff == 14 else 2
                        key.append(("Mag", "Client", r.delai - d_pfs, True, r.prix["Mag"]))
                        key.append(("PFS","Mag", d_pfs, False, r.cutoff, r.prix["PFS"]))
                        continue
                    if(r.depart_volu == "PFS"):
                        d_pfs = 1 if r.cutoff == 14 else 2
                        key.append(("Mag", "Client", r.delai - d_pfs, True, r.prix["Mag"]))
                        key.append(("PFS","Mag", d_pfs, True, r.cutoff, r.prix["PFS"]))
                        continue
                    if(r.depart_volu == "CAR"):
                        d_pfs = 1 if r.cutoff == 13 else 2
                        key.append(("Mag", "Client", r.delai - d_pfs, True, r.prix["Mag"]))
                        key.append(("PFS","Mag", d_pfs, False, r.cutoff, r.prix["PFS"]))
                        key.append(("CAR","Mag", r.prix["CAR"]))
                        continue

                if r.possible["n"]:
                    if(r.depart_volu == "Mag"):
                        print("cas non traité colis std PFS + volu depuis ",r.depart_volu)
                        continue
                    if(r.depart_volu == "PFS"):
                        key.append(("PFS","Client", r.delai, True, r.cutoff, r.prix["PFS"]))
                        continue
                    if(r.depart_volu == "CAR"):
                        print("cas non traité colis std PFS + volu depuis ",r.depart_volu)
                        continue
    for k in key:
        if trajet_dict.get(k) == None:
            trajet_dict[k] = 0
        trajet_dict[k] += r_count[last_r]
    stocks_volu_used = {}
    stocks_used = {}
    for l in L:
        stocks_volu_used[l] = pulp.value(pulp.lpSum([quantite_volu(pbDecat,i,n,h,d,r,l) for n in N for i in range(n+1) for h in H for d in D for r in R]) / pbDecat.nb_total_volu)
    stocks_used["PFS"] = pulp.value(PFS_stock/pbDecat.nb_total_std)
    stocks_used["Mag"] = pulp.value(Mag_stock/pbDecat.nb_total_std)
    for k in stocks_volu_used:
        stocks_volu_used[k] = round(stocks_volu_used[k],2)
    for k in stocks_used:
        stocks_used[k] = round(stocks_used[k],2)
    for k in cout_total_prep:
        cout_total_prep[k] = round(cout_total_prep[k],2)

    file = open(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S")) + ".txt", "w")
    file.write("nb cmd : " + str(n_total_cmd) + " --- partie volumineux : " +
            str(d_volumineuse) + "%\n\n")
    file.write("Stocks : " + str(stocks) + "\n")
    file.write("Stocks utilisés : " + str(stocks_used) + "\n\n")
    file.write("Stocks volumineux : " + str(stocks_volu) + "\n")
    file.write("Stocks volumineux utilisés : " + str(stocks_volu_used) + "\n\n")
    file.write("Coûts de préparation [standard, volumnieux] (par article) : " + str(prix_preparation) + "\n")
    file.write("Coûts totaux de préparation : " + str(cout_total_prep) + "\n\n")
    file.write("\n")
    file.write("{:20s}\t{}\t{}\t{}\t{}\t{}\n".format("trajet","J+n","cutoff","prix", "nb cmd", "coût total hors préparation\n"))

    for r in R:
        if r_count[r] != 0:
            file.write("{:<20s}\t{:3d}\t{:^7d}\t{:2.2f}\t {:5.0f}\t{:>6.0f} €\n".format(r.to_str(), r.delai, r.cutoff, r.prix_total(), r_count[r], int(r_count[r]) * r.prix_total()))


    file.write("\n")
    file.write("nb cmd  (départ, arrivée, j+n, volumineux, cutoff, prix) \t prix total \n")

    for k in sorted(trajet_dict):
        file.write("{:>5d}   {} {}\n".format(int(trajet_dict[k]), str(k), int(trajet_dict[k]) * k[-1]))

    file.write("\n")
    file.write("nombre de commande correct : " + str(abs(sum(r_count.values()) - int(n_total_cmd))<0.1) + "\n")
    file.write("Fonction objectif : {:,.2f} €".format(pulp.value(pbDecat.lp.objective)) + "\n")
    file.close()

In [13]:
def gen_and_solve():
    probleme = DecathlonProbleme()
    get_data_from_file(probleme.listeRoute)
    probleme.demande_std, probleme.demande_volu, probleme.nb_total_std, probleme.nb_total_volu = get_demande()
    probleme.lp, probleme.var_flot_std, probleme.var_flot_volu = generer_probleme_et_variables(probleme.listeRoute)

    cost_function(probleme)
    contrainte_capacite(probleme)
    contrainte_satisfaire_demande(probleme)
    probleme.lp.solve()
    print("Status:", pulp.LpStatus[probleme.lp.status])
    print(len(probleme.listeRoute))
    write_to_file(probleme)

def get_problem_no_cost():
    probleme = DecathlonProbleme()
    get_data_from_file(probleme.listeRoute)
    probleme.demande_std, probleme.demande_volu, probleme.nb_total_std, probleme.nb_total_volu = get_demande()
    probleme.lp, probleme.var_flot_std, probleme.var_flot_volu = generer_probleme_et_variables(probleme.listeRoute)
    contrainte_capacite(probleme)
    contrainte_satisfaire_demande(probleme)
    return probleme
    
def solving_writing(probleme):
    probleme.lp.solve()
    print("Status:", pulp.LpStatus[probleme.lp.status])
    write_to_file(probleme)

In [14]:
def modify_price(probleme:DecathlonProbleme, depart, old_price, new_price, volu):
    for r in probleme.listeRoute:
        if r.volu == volu and r.prix.get(depart) and r.prix[depart] == old_price:
            r.prix[depart] = new_price

In [26]:
# Initialisation des données
N = range(1,10) # Nombre d'articles dans une commande
prix_car_Mag = 1
car_cutoff = 13
delai_car_Mag = 1
d_volumineuse = 0.15
n_total_cmd = int(189000*0.14)
stocks = {"PFS":0.75, "Mag":0.3}
stocks_volu = {"PFS": 0.2, "Mag": 0.5, "CAR": 0.5}
prix_preparation = {"PFS": [1.86, 1.98], "Mag": [3.17, 3.37], "CAR": [0, 1.98]} # utilisation : prix_preparation["PFS"][True] -> prix de préparation d'un article volumineux


In [27]:
gen_and_solve()

Status: Optimal
45


AttributeError: 'NoneType' object has no attribute 'nb_total_std'